In [55]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from newsapi import NewsApiClient
from datetime import datetime, timedelta
import numpy as np
from numpy.random import seed
seed(1)
import requests
from tensorflow import random
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import hvplot.pandas
import plotly.express as px
random.set_seed(2)
#import nltk as nltk
#nltk.download('vader_lexicon')
#from nltk.sentiment.vader import SentimentIntensityAnalyzer
#analyzer = SentimentIntensityAnalyzer()
#from nltk.corpus import stopwords, reuters
import hvplot.pandas
%matplotlib inline

In [56]:
load_dotenv()

True

In [57]:
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")
#print(alpaca_api_key)

In [58]:
api_key = os.getenv("news_api")
#print(api_key)

In [59]:
newsapi = NewsApiClient(api_key=api_key)

In [60]:
#15Min interval
timeframe = "1D"
start_date = pd.Timestamp("2019-01-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-12-31", tz="America/New_York").isoformat()
#today = pd.Timestamp("2021-01-02", tz="America/New_York").isoformat()
ticker = ["AAPL"]
nasdaq_ticker = alpaca.get_barset(
    ticker,
    timeframe,
    start=start_date,
    end=end_date,
    limit=1000,
).df
nasdaq_ticker.dropna(inplace=True)
nasdaq_ticker.head(5)
#apple_ticker.dropna(inplace=True)

AAPL                                      
                             open      high     low     close    volume
time                                                                   
2019-01-02 00:00:00-05:00  154.89  158.8500  154.23  157.5225  27803621
2019-01-03 00:00:00-05:00  143.98  145.7200  142.00  142.2900  82418448
2019-01-04 00:00:00-05:00  144.53  148.5499  143.80  148.1200  51267225
2019-01-07 00:00:00-05:00  148.70  148.8300  145.90  147.9100  49982480
2019-01-08 00:00:00-05:00  149.56  151.8200  148.52  150.6300  36592406

In [61]:
tech_df = pd.DataFrame()
#tech_df["NASDAQ"] = nasdaq_ticker["NDAQ"]["close"]
tech_df["Apple"] = nasdaq_ticker["AAPL"]["close"]
#tech_df["Google"] = nasdaq_ticker["GOOGL"]["close"]
#tech_df["Facebook"] = nasdaq_ticker["FB"]["close"]
#apple_df = apple_df.pct_change().dropna()
tech_df.index = tech_df.index.date 
tech_df.dropna(inplace=True)
tech_df.head()

,Apple
2019-01-02,157.5225
2019-01-03,142.2900
2019-01-04,148.1200
2019-01-07,147.9100
2019-01-08,150.6300


In [62]:
tech_returns = tech_df.pct_change().dropna()
tech_returns.head()

,Apple
2019-01-03,-0.096700
2019-01-04,0.040973
2019-01-07,-0.001418
2019-01-08,0.018390
2019-01-09,0.018190


In [63]:
csv_path = Path("../csvs/Book1.csv/")
oil_data = pd.read_csv(csv_path, index_col="Date", parse_dates=True, infer_datetime_format=True)
oil_data.sort_index(ascending = True, inplace = True)
oil_data.head()

,"Cushing, OK WTI Spot Price FOB (Dollars per Barrel)"
Date,
1986-01-02,25.56
1986-01-03,26.00
1986-01-06,26.53
1986-01-07,25.85
1986-01-08,25.87


In [64]:
oil = pd.DataFrame()
oil["DPB"] = oil_data["Cushing, OK WTI Spot Price FOB (Dollars per Barrel)"]

In [65]:
oil.head()

,DPB
Date,
1986-01-02,25.56
1986-01-03,26.00
1986-01-06,26.53
1986-01-07,25.85
1986-01-08,25.87


In [66]:
csv_path = Path("../csvs/covid_cases.csv/")
covid_data = pd.read_csv(csv_path, index_col="Date", parse_dates=True, infer_datetime_format=True)
##covid_data.sort_index(ascending = True, inplace = True)
covid_data.tail()

,New Cases,7-Day Moving Avg,Historic Cases
Date,,,
2020-01-26,3,6,0
2020-01-25,3,5,0
2020-01-24,1,5,0
2020-01-23,2,5,0
2020-01-22,33,4,0


In [67]:
covid_df = pd.DataFrame()
covid_df["US_Covid_Cases"] = covid_data["New Cases"]
#covid_df["Date"] = covid_data["Date"]
covid_df.head()

,US_Covid_Cases
Date,
2021-07-02,15555
2021-07-01,17911
2021-06-30,14353
2021-06-29,14823
2021-06-28,8669


In [68]:
df = tech_df.join(oil, how="inner")
df.head()

,Apple,DPB
2019-01-02,157.5225,46.31
2019-01-03,142.2900,46.92
2019-01-04,148.1200,47.76
2019-01-07,147.9100,48.27
2019-01-08,150.6300,49.58


In [69]:
data = covid_df.join(df, how="inner")
#data = data.join(vaccine_df, how="inner")
data.tail()

,US_Covid_Cases,Apple,DPB
2020-01-28,1,317.7600,53.33
2020-01-27,1,308.9600,53.09
2020-01-24,1,318.3300,54.09
2020-01-23,2,319.2899,55.51
2020-01-22,33,317.7000,56.76


In [70]:
ewma = pd.DataFrame()
ewma["Apple"] = data["Apple"]
ewma["SMA10"] = data.Apple.rolling(window=10).mean()
ewma["EWMA10"] = data["Apple"].ewm(halflife=5).mean()
ewma.tail()

,Apple,SMA10,EWMA10
2020-01-28,317.7600,319.12262,316.410544
2020-01-27,308.9600,317.86462,315.446075
2020-01-24,318.3300,317.69362,315.819398
2020-01-23,319.2899,317.10161,316.268652
2020-01-22,317.7000,316.71361,316.453939


In [71]:
data = ewma.join(covid_df, how="inner")
data.dropna(inplace=True)

In [72]:
data.head()

,Apple,SMA10,EWMA10,US_Covid_Cases
2021-06-15,129.7400,132.32800,131.737022,12891
2021-06-14,130.4900,131.90200,131.530690,7833
2021-06-11,127.3521,131.32621,130.863333,14362
2021-06-10,126.1100,130.59721,130.126482,12350
2021-06-09,127.1600,129.94321,129.678089,21517


In [73]:
data_scaled = StandardScaler().fit_transform(data)
print(data_scaled[0:5])

[[-0.76645482 -0.74502525 -0.76474617 -0.81610335]
 [-0.75981788 -0.74891214 -0.76669233 -0.89242927]
 [-0.78758595 -0.75416573 -0.77298696 -0.79390576]
 [-0.79857761 -0.76081723 -0.77993708 -0.82426712]
 [-0.78928589 -0.76678442 -0.78416641 -0.68593583]]


In [74]:
pca = PCA(n_components=2)
data_pca = pca.fit_transform(data_scaled)

In [75]:
df_pca = pd.DataFrame(
    data=data_pca, columns=["principal component 1", "principal component 2"]
)
df_pca.head()

,principal component 1,principal component 2
0,-0.990562,-1.187820
1,-0.966156,-1.260196
2,-1.018585,-1.173667
3,-1.022487,-1.206921
4,-1.066585,-1.075897


In [76]:
pca.explained_variance_ratio_

array([0.79218232, 0.19044676])

In [77]:
pca = PCA(n_components=3)

# Get two principal components for the iris data.
data_pca = pca.fit_transform(shopping_scaled)

In [78]:
df_pca = pd.DataFrame(
    data=data_pca,
    columns=["principal component 1", "principal component 2", "principal component 3"],
)
df_pca.head()

,principal component 1,principal component 2,principal component 3
0,-0.990548,-1.187819,-0.025386
1,-0.966142,-1.260195,-0.018043
2,-1.018572,-1.173666,-0.035622
3,-1.022474,-1.206920,-0.039337
4,-1.066572,-1.075896,-0.026827


In [79]:
pca.explained_variance_ratio_

array([0.7921823 , 0.19044578, 0.01612334])

In [80]:
model = KMeans(n_clusters=6, random_state=0)

# Fit the model
model.fit(df_pca)

# Predict clusters
predictions = model.predict(df_pca)

# Add the predicted class columns
df_pca["class"] = model.labels_
df_pca.head()

,principal component 1,principal component 2,principal component 3,class
0,-0.990548,-1.187819,-0.025386,0
1,-0.966142,-1.260195,-0.018043,0
2,-1.018572,-1.173666,-0.035622,0
3,-1.022474,-1.206920,-0.039337,0
4,-1.066572,-1.075896,-0.026827,0


In [81]:
fig = px.scatter_3d(
    df_pca,
    x="principal component 3",
    y="principal component 2",
    z="principal component 1",
    color="class",
    symbol="class",
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()